Avec Docker, il est très facile de conteneuriser une application pour packager le code source et ses dépendances. Nous allons donc pouvoir conteneuriser l'API contenant le modèle.

<blockquote><p>🙋 <b>Ce que nous allons faire</b></p>
<ul>
    <li>Créer l'image Docker contenant l'API.</li>
    <li>Configurer le système pour exécuter automatiquement le conteneur sur l'instance Docker.</li>
</ul>
</blockquote>

<img src="https://media.giphy.com/media/XeAE4MvXVwOLZsZJWO/giphy.gif" />

## Création de l'image Docker

Nous allons construire l'image Docker qui va contenir l'API du modèle. Pour cela, nous devrions a priori ajouter une clé SSH au dépôt Cloud Source pour que l'instance `docker` puisse cloner le projet. Mais profitons de l'interaction entre les services de Google Cloud : il est possible de donner des droits d'accès automatiquement à certaines VMs pour, par exemple, authentifier les actions de clonage Git.

Créons une VM `docker`. En modifiant les informations de l'instance, nous pouvons définir le **Niveau d'accès** à *Définir l'accès pour chaque API*. Sous Cloud Source Repositoires, sélectionnons *Lecture seule*.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/docker_api1.png" />

Enregistrons les paramètres et démarrons l'instance. En s'y connectant en SSH, nous pouvons cloner le dépôt `purchase_predict_api` depuis <a href="https://source.cloud.google.com/" target="_blank">Cloud Source</a>. En cliquant sur le bouton pour cloner, copions la commande via **SDK Google Cloud**. La commande de clonage via SSH ne fonctionnera pas puisque nous n'avons pas configuré de clés.

 Sur le dépôt, il faut se positionner sur la branche `staging`, puisqu'il n'y a aucun fichier par défaut.

Retournons dans le répertoire local et ajoutons le fichier `Dockerfile`.

Ce fichier est décomposé en plusieurs étapes.

- L'installation des paquets nécessaires (comme `libgomp1` pour LightGBM) et du gestionnaire `pip`.
- L'ajout des fichiers sources dans le dossier `/app` sur l'image Docker.
- L'exécution en parallèle des 4 processus Flask avec `gunicorn`.

Une fois le fichier enregistré, nous pouvons construire l'image.

Nous pouvons ensuite exécuter un conteneur avec l'image construire.

Malheureusement, au bout de quelques secondes ... plusieurs erreurs apparaissent !

En effet, nous n'avons pas défini les **variables d'environnement** ! Il faut donc spécifier au conteneur Docker les variables telles que nous les avions définies dans le fichier `.env` par exemple. Pour cela, il est plus commode de créer un fichier `env.list` par exemple.

Pour passer l'ensembles des variables en paramètre au conteneur Docker, nous pouvons utiliser l'argument `--env-file`.

Les 4 exécutions de l'API doivent maintenant être opérationnelles dans le conteneur.

> ❓ Comment avons-nous pu récupérer le modèle depuis Cloud Storage alors qu'il n'y a pas de compte de service ?

En effet, nous n'avons pas spécifié de compte de service ici. C'est justement parce que nous sommes sur **une VM située dans le même projet que le bucket** que l'authentification s'effectue par défaut. Les instances de VM de Google Cloud ont déjà des comptes de service par défaut avec notamment un accès en lecture et écriture vers Cloud Storage. Ainsi, cela est automatiquement transmis au conteneur Docker. Par contre, si nous étions sur un serveur d'un autre projet Google Cloud ou d'un autre fournisseur Cloud, alors il aurait fallu mettre la clé d'un compte de service sur l'instance hôte, renseigner le chemin à cette clé dans la variable `GOOGLE_APPLICATION_CREDENTIALS` et transmettre cette variable d'environnement au conteneur Docker.

Testons notre API pour voir que tout s'est bien déroulé.

In [2]:
import os
import requests
import pandas as pd

dataset = pd.read_csv(os.path.expanduser("~/data/primary.csv"))
dataset = dataset.drop(["user_session", "user_id", "purchased"], axis=1)

In [ ]:
requests.post(
    "http://xx.xx.xx.xx/predict",  # Remplacer par l'adresse IP de l'instance Docker
    json=dataset.sample(n=10).to_json()
).json()

## Configuration du système

Nous sommes capable d'exécuter notre API à partir d'un conteneur Docker. Seulement, nous devons réaliser toutes ces étapes manuellement si l'on souhaite par exemple créer une autre VM pour l'API ou si l'on redémarre l'actuelle VM. Pour optimiser la configuration du système, nous allons mettre en place plusieurs composantes.

- L'image Docker va être hébergé vers un <a href="https://console.cloud.google.com/gcr/images" target="_blank">Container Registry</a> qui ne sera accessible que dans notre projet GCP (et non public).
- Nous allons utiliser le nom d'hôte de l'instance MLflow plutôt que son adresse IP : en effet, en cas de redémarrage de cette instance, l'adresse IP publique, étant éphémère par défaut, sera modifiée. Ainsi, il faut modifier **toutes les références** de cette adresse IP dans les applications qui l'utilisent. Le nom d'hôte, quant à lui, permettra de faire référence à cette VM même en cas de redémarrage.
- Un service système sera crée pour exécuter automatiquement le conteneur contenant l'API.

### Registre de conteneurs Google Cloud

Dirigeons-nous vers le <a href="https://console.cloud.google.com/gcr/images" target="_blank">Container Registry</a> et créons un nouveau registre comme nous l'avions fait avec DockerHub.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/docker_api2.png" />

Comme nous pouvons le voir, il n'y a aucun registre pour l'instant. Mais contrairement à DockerHub, il n'est pas possible d'en créer un directement via l'interface : les registres sont automatiquement crées lorsqu'une image est envoyée via l'API Google Cloud.

Arrêtons la VM Docker et attribuons lui un nouvel accès API au stockage.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/docker_api3.png" />

Maintenant, nous disposons des droits d'accès pour envoyer une image vers un conteneur de notre projet. Toujours en SSH, après redémarrage et connexion à l'instance Docker, exécutons les commandes `gcloud` pour s'authentifier.

L'utilisation de `sudo` est importante, car cela va créer les fichiers de configuration dans `/root`, car Docker est utilisé avec `sudo`. Avant d'envoyer l'image vers le registre, attribuons-lui un tag permettant de faire référence à notre projet Google Cloud.

Il ne reste plus qu'à envoyer l'image vers le registre.

Le registre est bien crée avec l'image.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/docker_api4.png" />

### Nom de domaine du serveur MLflow

Avant de configurer les fichiers `systemd` pour exécuter automatiquement le conteneur sur la machine, récupérons le nom de domaine de l'instance MLflow. Après connexion SSH, nous pouvons simplement utiliser la commande suivante.

Si l'on retourne sur l'instance Docker, en SSH, nous pouvons faire un `ping` pour vérifier que le nom de domaine correspond bien à l'instance MLflow.

À noter que ce nom de domaine n'est accessible **qu'à l'intérieur du projet Google Cloud** : la VM ne sera pas joignable depuis son propre ordinateur.

### Configuration du `systemd`

La dernière étape consiste à créer un service `systemd` qui permettra d'exécuter automatiquement le conteneur en arrière-plan tout en garantissant le redémarrage. Mais avant, rappelons-nous que les variables d'environnements doivent être configurés, et avec un `systemd`, il n'est pas possible faire des `export`.

Pour pouvoir configurer les variables d'environnements pour un service, il faut les centraliser dans un fichier de configuration, que nous allons créer avec `sudo nano /etc/default/purchase_predict_api`.

Il ne reste plus qu'à créer le fichier `systemd`. Ce fichier concentre trois blocs que nous allons expliciter.

- **Unit** fait référence aux unités qui doivent être au préalable en cours d'exécution pour que ce service puisse être lancé. En l'occurence, les services réseaux et de gestion de fichier doivent être lancés pour Zookeeper.
- **Service** contient les informations du service.
- **Install** spécifie la méthode d'installation du service.

On pourra trouver <a href="https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/7/html/system_administrators_guide/chap-managing_services_with_systemd" target="_blank">plus d'informations ici</a> pour les fichiers `systemd`.

Après avoir enregistré le fichier, nous activons le service et l'ajoutons au services systèmes à démarrer automatiquement.

Puis démarrons le service.

Pour vérifier si le conteneur est bien exécuté, nous pouvons vérifier que le port $80$ est bien utilisé.

Si le port $80$ n'est pas utilisé, il se peut que le conteneur ne soit pas en cours d'exécution. Pour cela, il est possible d'inspecter les sorties du système en affichant par exemple les 100 dernières lignes.

Testons une nouvelle fois l'API.

In [9]:
requests.post(
    "http://xx.xx.xx.xx/predict",  # Remplacer par l'adresse IP de l'instance Docker
    json=dataset.sample(n=10).to_json()
).json()

Redémarrons l'instance. A priori, si nous avons correctement configuré le `systemd`, le conteneur devra s'exécuter automatiquement au démarrage de l'instance. Après quelques secondes, le temps que l'instance redémarre, nous pouvons exécuter à nouveau la cellule ci-dessus.

<div class="alert alert-block alert-warning">
    Puisque l'instance possède une adresse IP éphémère, il faudra probablement changer l'IP dans le cellule.
</div>

Une fois terminé, nous pouvons stopper l'instance puisque nous n'allons plus l'utiliser par la suite.

## ✔️ Conclusion

Notre API est maintenant pleinement déployée.

- Nous avons créer une image Docker pour l'API.
- Nous avons configuré le système pour automatiser l'exécution de l'API.

> ➡️ Malgré tout, jusqu'ici, le travail était principalement manuel. À partir de maintenant, nous allons pleinement intégrer l'approche MLOps en <b>automatisant le déploiement</b> des différents projets.